# Tarefas abandonadas com PySpark

Este projeto utiliza PySpark para identificar e analisar tarefas abandonadas ao longo do tempo, com o objetivo de monitorar padrões de abandono e apoiar a tomada de decisões para melhorias.

**Fluxo do projeto:**

1. **Extração e transformação:** Os dados são extraídos de um arquivo CSV, tratados e transformados em DataFrames Spark.
2. **Persistência:** Os dados processados são enviados e armazenados em uma tabela DynamoDB.
3. **Consulta e análise:** Os dados são recuperados do DynamoDB, processados em DataFrames e utilizados para gerar relatórios mensais dos últimos 6 meses, detalhando o volume de tarefas não concluídas por tipo.



## 1. Extração e transformação

In [ ]:
from src.clients.spark_client import get_spark_session
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, trim, when, lit,  current_timestamp, try_to_timestamp, date_format, concat, udf
import uuid


spark = get_spark_session()


base_dataframe = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .option('enconding', 'ISO-8859-1')
    .csv('./data/amostragem.csv')
)



df = base_dataframe.withColumnsRenamed(
    {'Nome da Tarefa': 'nome', 
     'Data de Criação': 'data', 
     'Data de Conclusão': 'data_conclusao', 
     'Status': 'status', 
     'Tipo da Tarefa': 'tipo_tarefa', 
     'ID do Usuário': 'user_id',
     'Usuário': 'usuario'
     }
    )

df = df.withColumn('status', df['status'].cast(StringType()))
df = df.withColumn('data', df['data'].cast(StringType()))
df = df.withColumn('data_conclusao', df['data_conclusao'].cast(StringType()))



Realiza a limpeza dos dados conforme os critérios abaixo:
- Exclui tarefas cujo nome está nulo ou vazio.
- Atribui a data atual para tarefas sem data ou com data inválida.

In [97]:

df = df.filter((df['usuario'] == 'Jeferson Klau') & (df['status'] != 3))
df = df.distinct()
df = df.withColumn('nome', trim('nome'))
df = df.withColumn('nome', when(col('nome') == '', lit(None)).otherwise(col('nome')))
df = df.dropna(subset=['nome'])

df = df.withColumn('data', when(try_to_timestamp(col('data')).isNotNull(), col('data')).otherwise(current_timestamp().cast(StringType())))



In [ ]:
df.show(100)

In [ ]:
Faz a transformação dos dados para o padrão esperado da tabela do DynamoDB

In [ ]:

def generate_uuid():
  return str(uuid.uuid4())

uuid_udf = udf(generate_uuid, StringType())
user_id = '035c6ada-4091-703a-1837-677cad18d4a5'

df = df.withColumn('status', when(df['status'] == 1, 'TODO').otherwise('DONE'))
df = df.withColumn('user_id', when(df['user_id'] == 'b4853fc1f03a3a4cec530a98a94d89ad', user_id))
df = df.withColumn('PK', concat(lit('LIST#'), date_format(col('data'), 'yyyyMMdd')))
df = df.withColumn('SK', concat(lit('ITEM#'), uuid_udf()))  
df = df.select(df.PK, df.SK, df.user_id, df.usuario, df.nome, df.data, df.data_conclusao, df.status, df.tipo_tarefa)



## 2. Enviando os dados para a tabela do dynamo

In [100]:
import time
from src.clients.dynamodb_client import DynamoDBClient

TAMANHO_LOTE = 100
DELAY_EM_SEGUNDOS = 2


dynamodb = DynamoDBClient()
table = dynamodb.get_table('teste')

pandas_df = df.toPandas()
rows = pandas_df.to_dict(orient='records')

for i in range(0, len(rows), TAMANHO_LOTE):
  batch_rows = rows[i:i+TAMANHO_LOTE]

  with table.batch_writer() as batch:
    for item in batch_rows:
      
      batch.put_item(Item=item) 

  time.sleep(DELAY_EM_SEGUNDOS)

print('Todos items foram inseridos')

Todos items foram inseridos


## 3. Obtendo volume de tarefas abandonadas 

In [101]:
from boto3.dynamodb.conditions import Key
from pyspark.sql.types import StringType, StructField, StructType

# Query para obter as tarefas não concluídas
response = table.query(
    IndexName='user_id-PK-index',
    KeyConditionExpression=Key('user_id').eq(user_id) & Key('PK').begins_with('LIST#'),
    FilterExpression=Key('status').eq('TODO')
)

items = response['Items']

schema = StructType([
    StructField('PK', StringType(), False),
    StructField('SK', StringType(), False),
    StructField('user_id', StringType(), False),
    StructField('usuario', StringType(), False),
    StructField('nome', StringType(), False),
    StructField('data', StringType(), False),
    StructField('data_conclusao', StringType(), True),
    StructField('status', StringType(), False),
    StructField('tipo_tarefa', StringType(), False)
])

dynamodb_dataframe = spark.createDataFrame(items, schema=schema)




In [102]:
import pyspark.sql.functions as F

def obter_abandonadas(data):
    abandonadas_df = (
        dynamodb_dataframe
        .select(
            F.col('nome'),
            F.col('data'),
            F.col('tipo_tarefa')
        
        )
        .where(
            ((F.col('tipo_tarefa') == 'Tarefa a Ser Feita') & (F.datediff(F.lit(data), F.col('data')) >= 15)) |
            ((F.col('tipo_tarefa') == 'Item de Compra') & (F.datediff(F.lit(data), F.col('data')) >= 30)) 
        ).orderBy('tipo_tarefa')
    )

    return abandonadas_df



In [ ]:
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta

today = date.today()
datas = [(today - relativedelta(months=i)) for i in range(5, -1, -1)]

resultados = [
    obter_abandonadas(data)
    .groupby(F.col('tipo_tarefa'))
    .count().toPandas()
    .rename(columns={'count': data.strftime('%Y-%m')}) 
    .set_index('tipo_tarefa')
    for data in datas
    ]

abandonadas_por_data = pd.concat(resultados, axis=1)
abandonadas_por_data.to_csv('abandonadas_por_data.csv', index_label='', sep=',')
    

